<a href="https://colab.research.google.com/github/alessandronascimento/pyLiBELa/blob/main/pyLiBELa_SB2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Downloading dependencies {display-mode: "form"}

%%capture
! apt-get install python-dev-is-python3 zlib1g-dev libeigen3-dev libgsl-dev libnlopt-cxx-dev libgsl-dev
!pip install ipython-autotime
%load_ext autotime
!pip3 install condacolab
import condacolab
condacolab.install()
!mamba install openbabel

time: 1min 1s (started: 2023-09-05 18:40:47 +00:00)


In [1]:
#@title Installing pyLiBELa {display-mode: "form"}

%%capture
! git clone https://github.com/alessandronascimento/pyLiBELa.git
! mv pyLiBELa/src src
! rm -rf pyLiBELa
! mkdir -p obj
! rm -f Makefile*
! wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/main/Colabs/Makefile
! make -j2

In [2]:
#@title Importing pyLiBELa {display-mode: "form"}
try:
  from pyPARSER import *
  from pyMol2 import *
  from pyWRITER import *
  from pyGrid import *
  from pyCOORD_MC import *
  from pyFindHB import *
  from pyEnergy2 import *
  from pyGaussian import *
  from pyConformer import *
  from pyRAND import *
  from pyMcEntropy import *
  from pySA import *
  from pyOptimizer import *
  from pyMC import *
  from pyFullSearch import *
  from pyDocker import *
  print('pyLiBELa is imported!')
except ImportError:
  print('An ImportError occurred, try running this cell again!')

pyLiBELa is imported!


In [3]:
#@title Getting SB2021 data {display-mode: "form"}

%%capture
from google.colab import drive
drive.mount('/content/drive/')
sb_folder = '/content/drive/MyDrive/pyLiBELa/SB/' #@param {type:"string"}

with open(sb_folder+'list.txt') as f:
    targets_list = f.readlines()


targets_list = [target[0:4] for target in targets_list]


In [4]:
#@title Organizing test files {display-mode: "form"}
i=0
target=targets_list[i]
folder = sb_folder+target
%cd $folder
! mkdir test
%cd test
! cp ../*lig.am1bcc.mol2.gz lig.mol2.gz
! cp ../*.rec.clean.mol2.gz rec.mol2.gz
! obabel -imol2 lig.mol2.gz -osmi -O lig.smi

/content/drive/MyDrive/pyLiBELa/SB/121P
mkdir: cannot create directory ‘test’: File exists
/content/drive/MyDrive/pyLiBELa/SB/121P/test
1 molecule converted


In [5]:
#@title Docking parameters {display-mode: "form"}

import os
import timeit
import numpy as np

Input = PARSER()

# Some default options:
# 1. We are using two processors for Grid calculations;
#

Input.generate_conformers = True;
Input.dock_parallel = True;
Input.parallel_jobs = 2;
Input.write_grids = True
Input.use_grids = True
Input.write_mol2 = True
Input.atom_limit = 60 #@param {type:"number"}
    #atom limit not counting H

# Energy Calculation Parameters:
scoring_function = "0" #@param ["0", "1", "2", "3"]
Input.dielectric_model = "r" #@param ["r", "constant"]
Input.scoring_function = int(scoring_function)
grid_dimension = 30.0 #@param {type:"number"}
Input.grid_spacing = 0.5 #@param {type:"number"}
Input.solvation_alpha = 0.1 #@param {type:"number"}
Input.solvation_beta = -0.005 #@param {type:"number"}

# Optimization parameter:
Input.min_tol = 1E-10;
Input.min_delta = 1E-5;
Input.dock_min_tol = 1E-10;
search_box = 20.0 #@param {type:"number"}
Input.timeout = 30 #@param {type:"number"}
Input.min_timeout = 30 #@param {type:"number"}
Input.overlay_optimizer = "mma" #@param ["mma", "ln_auglag", "subplex", "none"]
Input.energy_optimizer = "mma" #@param ["direct", "isres", "crs", "esch", "stogo", "mma", "simplex", "none"]
if (Input.scoring_function < 3):
  delta = 2.5 #@param {type:"number"}
  Input.deltaij6 = (delta*delta*delta*delta*delta*delta)
  delta_es = 2.5 #@param {type:"number"}
  Input.deltaij_es6 = pow(delta_es, 6);
  Input.deltaij_es3 = (delta_es*delta_es*delta_es)

Input.search_box_x, Input.search_box_y, Input.search_box_z = search_box, search_box, search_box;
Input.x_dim, Input.y_dim, Input.z_dim = grid_dimension, grid_dimension, grid_dimension;

In [6]:
tag  = 'A' #@param {type:"string"}
Input.grid_prefix = "McGrid_" + target + '_' + tag


insmiles = open('lig.smi', 'r');
line=insmiles.readline().strip()
line2 = line.split()
ligand_smiles = line2[0]
insmiles.close();
print('SMILES read: %s' % ligand_smiles)

Lig2 = Mol2()

lig_src = 'lig.mol2.gz'
rec_src = 'rec.mol2.gz'

REC = Mol2(Input, rec_src)
RefLig = Mol2(Input, lig_src)
Lig2.parse_smiles(Input, ligand_smiles, 'LIG')

print('Receptor and reference ligand parsed successfully!')
print("Receptor has %4d atoms." % REC.N)
print('Reference Ligand has %4d atoms' % RefLig.N)
print('Search ligand has %4d atoms' % Lig2.N)

SMILES read: [P]([O])([O])([O])C[P]([O])([O])O[P]([O])([O])OC[C@H]1O[C@H]([C@@H]([C@@H]1O)O)N1C=NC2=C1N=C(NC2=O)N
Receptor and reference ligand parsed successfully!
Receptor has 2620 atoms.
Reference Ligand has   46 atoms
Search ligand has   46 atoms


In [ ]:
#@title Grid Generation and Docking calculation
!rm -rf McLiBELa_dock.mol2.gz
import timeit

Writer = WRITER(Input)
Coord = COORD_MC()
HB = FindHB()
Ene  = Energy2(Input)

for i in range(len(REC.residue_pointer)-1):
  HB.parse_residue(REC.residue_pointer[i]-1, REC.residue_pointer[i+1]-2, REC.resnames[i], REC, RefLig, 9.0);
HB.find_ligandHB(lig_src, RefLig);
print('The receptor has %5d / %5d HB donors/acceptors around the active site.' % (len(REC.HBdonors), len(REC.HBacceptors)));

Dock = Docker(Writer)
center = Coord.compute_com(RefLig)

print()
start_energy = Ene.compute_ene(REC, RefLig, RefLig.xyz);
print('Starting energy: %7.3f kcal/mol' % start_energy);
print()
print('Generating grids. This may take a while..')

start_time = timeit.default_timer()
Grids = Grid(Input, Writer, REC, center)
time = timeit.default_timer() - start_time

time_per_atom = time/REC.N
print('Grids computed, and it took %.2f s! That means %.4f s per atom.' %(time,time_per_atom))
grid_energy = Ene.compute_ene(Grids, RefLig, RefLig.xyz);
print('Grid original energy: %7.3f kcal/mol' % grid_energy);
print('Grid error: %7.3f' % abs(100.*(start_energy-grid_energy)/start_energy));
print()
print()
print('Starting docking calculation...')
Dock.run(REC, Lig2, RefLig, center, Input, Grids, 0)
print('Docking calculation finished!')

Writer.write_box(center, Grids.xbegin, Grids.ybegin, Grids.zbegin, Grids.xend, Grids.yend, Grids.zend)


In [ ]:
grid_name = Input.grid_prefix + '.grid'
!mv $grid_name test/
log_name = 'McLiBELa' + target + '_' + tag + '.log'


!mv McLiBELa.log test/$log_name


In [ ]:
!tail McLiBELa.log
rmsd = 0
use_cuda = 'False'
log_add = 'Use CUDA = %s \n Grids Time = %.2f \n Time per atom  = %.2f \n RMSD = %.2f'%(use_cuda,time,time_per_atom,rmsd)
print(log_add)
#!sed -i
#printf("* %-30s %-66d*\n", "number_of_conformers", Input->lig_conformers);